# Tutorial: Implement Agent Orchestration Loop in Notebook

Goals:
- Implement a manual orchestration loop directly in notebook cells.
- Compare manual-loop behavior against the production LangGraph workflow.
- Observe `continue/replan/end` transitions under both CodeAct and ReAct.

## Outline
1. Setup runtime components.
2. Manual orchestration loop (educational version).
3. Official LangGraph workflow run.
4. Compare outcomes.


In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path


def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists() and (candidate / "src").exists():
            return candidate
    raise RuntimeError("Could not locate repo root")


REPO_ROOT = find_repo_root(Path.cwd())
if str(REPO_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(REPO_ROOT / "src"))

from manus_three_agent.agents import ArchitectAgent, CriticAgent, WorkerAgent
from manus_three_agent.core.schemas import PlanStep
from manus_three_agent.core.state import build_initial_state
from manus_three_agent.core.types import ModelConfig
from manus_three_agent.environments.simulator import GenericSimulatorEnvironment
from manus_three_agent.graph import build_workflow
from manus_three_agent.prompts import PromptTemplates
from manus_three_agent.tools import build_default_tool_registry
from manus_three_agent.tracing import TraceCollector

prompts = PromptTemplates(config_dir=str(REPO_ROOT / "configs" / "prompts"))
model_cfg = ModelConfig(model="mock")
tracer = TraceCollector.disabled()
tools = build_default_tool_registry()

print(f"Repo root: {REPO_ROOT}")


## Step 1: Manual orchestration loop

In [ ]:
def run_manual_loop(goal: str, agentic_mode: str = "codeact", max_steps: int = 6):
    env = GenericSimulatorEnvironment()
    architect = ArchitectAgent(model_cfg, prompts, tracer=tracer, force_mock=True, agentic_mode=agentic_mode)
    worker = WorkerAgent(model_cfg, prompts, tools, tracer=tracer, force_mock=True, agentic_mode=agentic_mode)
    verifier = CriticAgent(model_cfg, prompts, tracer=tracer, force_mock=True, agentic_mode=agentic_mode)

    state = build_initial_state(
        goal=goal,
        observation=env.reset(goal=goal),
        max_steps=max_steps,
        dynamic_replanning=True,
        use_cot=False,
        agentic_mode=agentic_mode,
    )

    timeline = []

    while not state["done"] and state["step_count"] < state["max_steps"]:
        if not state["plan"] or state["current_step_idx"] >= len(state["plan"]):
            plan_out = architect.plan(
                goal=state["goal"],
                observation=state["observation"],
                action_history=state["action_history"],
                use_cot=state["use_cot"],
                step=state["step_count"],
            )
            state["plan"] = [s.model_dump() for s in plan_out.steps]
            state["current_step_idx"] = 0
            timeline.append({"phase": "planner", "step": state["step_count"], "plan_len": len(state["plan"])})

        current_step = PlanStep.model_validate(state["plan"][state["current_step_idx"]])
        action = worker.execute(
            goal=state["goal"],
            plan_step=current_step,
            observation=state["observation"],
            step_index=state["current_step_idx"],
            total_steps=len(state["plan"]),
            use_cot=state["use_cot"],
            step=state["step_count"],
        )

        env_result = env.step(action=action, step_count=state["step_count"] + 1)
        state["action_history"].append(action.model_dump())
        state["observation"] = env_result.observation
        state["step_count"] += 1
        state["current_step_idx"] += 1

        if action.is_final or env_result.done:
            state["done"] = True
            state["success"] = True
            state["final_answer"] = action.final_answer or env_result.final_answer

        review = verifier.review(
            goal=state["goal"],
            observation=state["observation"],
            action_history=state["action_history"],
            current_step_idx=state["current_step_idx"],
            plan_length=len(state["plan"]),
            step=state["step_count"],
        )
        state["review_history"].append(review.model_dump())
        state["decision"] = review.decision

        timeline.append({
            "phase": "worker+verifier",
            "step": state["step_count"],
            "action_summary": action.summary,
            "decision": review.decision,
        })

        if state["done"]:
            break
        if review.decision == "replan":
            state["plan"] = []
            state["current_step_idx"] = 0
        elif review.decision == "end":
            state["done"] = True
            state["success"] = state["success"] or review.should_succeed
            if not state["final_answer"]:
                state["final_answer"] = "Verifier ended run."

    return state, timeline


goal = "Build a compact long-horizon agent evaluation routine"
manual_results = {}
for mode in ["codeact", "react"]:
    final_state, timeline = run_manual_loop(goal=goal, agentic_mode=mode)
    manual_results[mode] = {
        "summary": {
            "success": final_state["success"],
            "step_count": final_state["step_count"],
            "decision": final_state["decision"],
            "final_answer": final_state["final_answer"],
        },
        "timeline": timeline,
    }

print(json.dumps(manual_results, indent=2, ensure_ascii=False))


## Step 2: Run official LangGraph workflow

In [ ]:
def run_graph_workflow(goal: str, agentic_mode: str = "codeact"):
    env = GenericSimulatorEnvironment()
    architect = ArchitectAgent(model_cfg, prompts, tracer=tracer, force_mock=True, agentic_mode=agentic_mode)
    worker = WorkerAgent(model_cfg, prompts, tools, tracer=tracer, force_mock=True, agentic_mode=agentic_mode)
    verifier = CriticAgent(model_cfg, prompts, tracer=tracer, force_mock=True, agentic_mode=agentic_mode)

    workflow = build_workflow(architect, worker, verifier, env, tracer)
    initial_state = build_initial_state(
        goal=goal,
        observation=env.reset(goal=goal),
        max_steps=6,
        dynamic_replanning=True,
        use_cot=False,
        agentic_mode=agentic_mode,
    )
    return workflow.invoke(initial_state)

graph_results = {}
for mode in ["codeact", "react"]:
    out = run_graph_workflow(goal=goal, agentic_mode=mode)
    graph_results[mode] = {
        "success": out["success"],
        "step_count": out["step_count"],
        "final_answer": out["final_answer"],
        "decision": out["decision"],
    }

print(json.dumps(graph_results, indent=2, ensure_ascii=False))


## Step 3: Compare manual loop vs graph output

In [ ]:
comparison = {}
for mode in ["codeact", "react"]:
    comparison[mode] = {
        "manual_step_count": manual_results[mode]["summary"]["step_count"],
        "graph_step_count": graph_results[mode]["step_count"],
        "manual_success": manual_results[mode]["summary"]["success"],
        "graph_success": graph_results[mode]["success"],
    }

print(json.dumps(comparison, indent=2, ensure_ascii=False))


## Takeaways
- The manual loop helps architects reason about transition logic before optimization.
- The graph workflow is the production runtime path and should keep deterministic routing.
- Tracing can be attached to both manual and graph flows for trajectory-quality comparison.